In [1]:
import pandas as pd
import random
from nltk.tokenize import TweetTokenizer
from nltk import classify
from nltk import FreqDist
from nltk import MaxentClassifier
from nltk import confusionmatrix
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
# Download annotated comments and annotations.
# If you're Tracy, Courtney, or Amandalynne, don't run this step
# because you already have the data! If you aren't us, you will
# probably need to do this step.
# It will take a while.
ANNOTATED_COMMENTS_URL = 'https://ndownloader.figshare.com/files/7038044'
ANNOTATIONS_URL = 'https://ndownloader.figshare.com/files/7383751'


def download_file(url, fname):
    urllib.request.urlretrieve(url, fname)

# not needed if files already exist in directory
#download_file(ANNOTATED_COMMENTS_URL, 'attack_annotated_comments.tsv')
#download_file(ANNOTATIONS_URL, 'attack_annotations.tsv')

In [3]:
# Read the data into a Pandas dataframe.
comments = pd.read_csv('../attack_annotated_comments.tsv', sep='\t', index_col=0)
annotations = pd.read_csv('../attack_annotations.tsv',  sep='\t')

# Label a comment as an attack if over half of annotators did so.
# We can tinker with this threshold later.
labels = annotations.groupby('rev_id')['attack'].mean() > 0.5

# Join labels and comments
comments['attack'] = labels

In [4]:
print('preprocessing...')

# Preprocess the data -- remove newlines, tabs, quotes
# Something to consider: remove Wikipedia style markup (::'s and =='s)
comments['comment'] = comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("`", " "))

preprocessing...


In [5]:
# Very small data set for debugging
# train_data = comments.iloc[0:100]
# test_data = comments.iloc[200:210]

# Grab the training data (seems to be 60%)
train_data = comments.loc[comments['split'] == 'train']
dev_data = comments.loc[comments['split'] == 'dev']
test_data = comments.loc[comments['split'] == 'test']

In [6]:
# The list of gold-standard labels for the data
train_labels = train_data["attack"].tolist()
dev_labels = dev_data["attack"].tolist()
test_labels = test_data["attack"].tolist()
# Put all the training data (comments) into a list
train_texts = train_data["comment"].tolist()
dev_texts = dev_data["comment"].tolist()
test_texts = test_data["comment"].tolist()

In [7]:
def randomly_sample(comments, labels):
    # grab all the attacks to see how many we need to match
    attack_indices = [i for i in range(len(comments)) if labels[i] == True]
    new_training = [comments[i] for i in attack_indices]
    new_labels = [labels[i] for i in attack_indices]

    # grab all the ones that are not attacks, shuffle them and select the same number of non-attacks
    non_attack_indices = [i for i in range(len(comments)) if labels[i] == False]
    random.shuffle(non_attack_indices)
    for i in range(len(attack_indices)):
        new_training.append(comments[i])
        new_labels.append(labels[i])

    # shuffle the training and labeling so that they're still matched 1:1
    shuf_indices = [i for i in range(len(new_training))]
    random.shuffle(shuf_indices)
    shuffled_training = [new_training[i] for i in shuf_indices]
    shuffled_labels = [new_labels[i] for i in shuf_indices]

    return shuffled_training, shuffled_labels

In [8]:
train_texts, train_labels = randomly_sample(train_texts, train_labels)
dev_texts, dev_labels = randomly_sample(dev_texts, dev_labels)
test_texts, test_labels = randomly_sample(test_texts, test_labels)

In [36]:
print('creating feature sets...')
from nltk.util import ngrams

# word frequency features
def char_freq(words):
    bigram = ngrams(list(words), 2)
    trigram = ngrams(list(words), 3)
    fourgram = ngrams(list(words), 4)
    fivegram = ngrams(list(words), 5)
    
    ngram = list(words)
    ngram.extend([ "%s%s" % x for x in bigram ])
    ngram.extend([ "%s%s%s" % x for x in trigram ])
    ngram.extend([ "%s%s%s%s" % x for x in fourgram ])
    ngram.extend([ "%s%s%s%s%s" % x for x in fivegram ])
    return dict(FreqDist(ngram)) 

# Create the training set and test set
train_features = [char_freq(x) for x in train_texts]
train_set = list(zip(train_features, train_labels))
test_features = [char_freq(x) for x in test_texts]
test_set = list(zip(test_features, test_labels))

creating feature sets...


In [37]:
from sklearn.feature_extraction import DictVectorizer

features = train_features+test_features
vec = DictVectorizer()
feats = vec.fit_transform(features)

In [38]:
train_feats = feats[:len(train_texts), :]
test_feats = feats[len(train_texts):, :]

assert len(train_features) == train_feats.shape[0]
assert len(test_features) == test_feats.shape[0]

In [39]:
print('classifying...')

model = LogisticRegression()
model.fit(train_feats, train_labels)

classifying...


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [40]:
sys_prob = model.predict_proba(test_feats)
sys_prob = sys_prob[:, 1:2]
sys_labels = model.predict(test_feats)

In [41]:
# Extract Attack probabilities for AUROC measure
accuracy = model.score(train_feats, train_labels)
roc_auc = metrics.roc_auc_score(test_labels, sys_prob)
fpr, tpr, threshold = metrics.roc_curve(test_labels, sys_prob)
precision = metrics.precision_score(test_labels, sys_labels)
recall = metrics.recall_score(test_labels, sys_labels)

# Evaluation metrics
f1 = metrics.f1_score(test_labels, sys_labels)
print('\nAccuracy: {}\nF1: {}\nAUC: {}\nPrecision: {}\nRecall: {}'.format(accuracy, f1, roc_auc, precision, recall))


Accuracy: 0.9266617155588563
F1: 0.88478655018955
AUC: 0.9332323373173971
Precision: 0.8785597381342062
Recall: 0.8911022576361222


In [55]:
# Print confusion matrix
cm = confusionmatrix.ConfusionMatrix(test_labels, sys_labels)
print(cm.pretty_format(sort_by_count=True, show_percents=True, truncate=9))

      |             F |
      |      T      a |
      |      r      l |
      |      u      s |
      |      e      e |
------+---------------+
 True | <45.5%>  9.1% |
False |  16.6% <28.7%>|
------+---------------+
(row = reference; col = test)



In [17]:
# Plot the ROC
def plot_ROC(fpr, tpr, roc_auc):
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.savefig("MaxEnt_char_roc.png")
    
plot_ROC(fpr, tpr, roc_auc)